In [1]:
!pip install geopandas shapely openpyxl folium


In [2]:
# Import all required libraries
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import numpy as np
import matplotlib.pyplot as plt
import folium
from google.colab import files
import warnings
warnings.filterwarnings('ignore')

print("✓ All libraries imported successfully!")

✓ All libraries imported successfully!


In [3]:
# Upload the shapefile components
print("Please upload ALL 6 shapefile files for statistical areas:")
print("- statistical_areas_2022.shp")
print("- statistical_areas_2022.shx")
print("- statistical_areas_2022.dbf")
print("- statistical_areas_2022.prj")
print("- statistical_areas_2022.cpg")
print("- statistical_areas_2022.xml")
print()

uploaded = files.upload()

Please upload ALL 6 shapefile files for statistical areas:
- statistical_areas_2022.shp
- statistical_areas_2022.shx
- statistical_areas_2022.dbf
- statistical_areas_2022.prj
- statistical_areas_2022.cpg
- statistical_areas_2022.xml



Saving statistical_areas_2022.cpg to statistical_areas_2022.cpg
Saving statistical_areas_2022.dbf to statistical_areas_2022.dbf
Saving statistical_areas_2022.prj to statistical_areas_2022.prj
Saving statistical_areas_2022.shp to statistical_areas_2022.shp
Saving statistical_areas_2022.shx to statistical_areas_2022.shx
Saving statistical_areas_2022.xml to statistical_areas_2022.xml


In [4]:
# Load statistical areas shapefile
gdf_areas = gpd.read_file('statistical_areas_2022.shp')

print(f"✓ Loaded {len(gdf_areas)} statistical areas")
print(f"✓ CRS: {gdf_areas.crs}")
print(f"✓ Statistical area codes: {sorted(gdf_areas['STAT_2022'].unique())}")

✓ Loaded 25 statistical areas
✓ CRS: EPSG:2039
✓ Statistical area codes: [np.int32(11), np.int32(12), np.int32(13), np.int32(14), np.int32(15), np.int32(16), np.int32(17), np.int32(18), np.int32(19), np.int32(21), np.int32(22), np.int32(23), np.int32(24), np.int32(25), np.int32(26), np.int32(31), np.int32(32), np.int32(33), np.int32(41), np.int32(42), np.int32(43), np.int32(44), np.int32(45), np.int32(46), np.int32(47)]


In [5]:
# Upload Coffee Shops CSV file
print("Please upload Eilat_coffeShops_data_2025.csv")
uploaded = files.upload()

# Load Coffee Shops data
coffee_shops = pd.read_csv('Eilat_coffeShops_data_2025.csv')

print(f"\n✓ Loaded {len(coffee_shops)} coffee shop records")
print(f"\nColumns: {coffee_shops.columns.tolist()}")
print("\nFirst few rows:")
print(coffee_shops.head(10))

# Check data types
print("\n" + "="*80)
print("Data types:")
print(coffee_shops.dtypes)

Please upload Eilat_coffeShops_data_2025.csv


Saving Eilat_coffeShops_data_2025.csv to Eilat_coffeShops_data_2025.csv

✓ Loaded 339 coffee shop records

Columns: ['cid', 'title', 'description', 'categoryName', 'totalScore', 'temporarilyClosed', 'permanentlyClosed', 'url', 'website', 'street', 'location/lat', 'location/lng', 'Activity times']

First few rows:
                    cid                                  title description  \
0  13304296502578382367            Ilan's The Alternative Cafe         NaN   
1  12043325052248165199              Cafe Orca - Marina Divers         NaN   
2  10214898044927479307                           DESERT & SEA         NaN   
3   2125374989964811871         Haoman 9- Coffee Confectionery         NaN   
4   6409751934090735763               Eilat wines | יינות אילת         NaN   
5  10391316705344374484                 Uga Chaga Bakery Eilat         NaN   
6    273177842584156848                    COFFEE BAR SCORPION         NaN   
7   8637314661383373859  Café LOBBY at Shoshana קפה לובי שושנ

In [6]:
print("Examining coffee shops data...\n")
print("="*80)

# Display sample data
print("Sample data:")
print(coffee_shops.head(10))

# Check for missing coordinates
missing_coords = coffee_shops[
    coffee_shops['location/lat'].isna() |
    coffee_shops['location/lng'].isna()
]

print("\n" + "="*80)
print(f"Records with missing coordinates: {len(missing_coords)}")
print(f"Records with valid coordinates: {len(coffee_shops) - len(missing_coords)}")

# Check coordinate ranges (should be in Eilat area)
print("\n" + "="*80)
print("Coordinate ranges:")
print(f"  Latitude range: {coffee_shops['location/lat'].min():.6f} to {coffee_shops['location/lat'].max():.6f}")
print(f"  Longitude range: {coffee_shops['location/lng'].min():.6f} to {coffee_shops['location/lng'].max():.6f}")

Examining coffee shops data...

Sample data:
                    cid                                  title description  \
0  13304296502578382367            Ilan's The Alternative Cafe         NaN   
1  12043325052248165199              Cafe Orca - Marina Divers         NaN   
2  10214898044927479307                           DESERT & SEA         NaN   
3   2125374989964811871         Haoman 9- Coffee Confectionery         NaN   
4   6409751934090735763               Eilat wines | יינות אילת         NaN   
5  10391316705344374484                 Uga Chaga Bakery Eilat         NaN   
6    273177842584156848                    COFFEE BAR SCORPION         NaN   
7   8637314661383373859  Café LOBBY at Shoshana קפה לובי שושנה         NaN   
8  16582584323208051397                       בייקרי קופי מרקט         NaN   
9  15185075342429951591                          Crema Express         NaN   

  categoryName  totalScore  temporarilyClosed  permanentlyClosed  \
0         Cafe         3.2  

In [7]:
print("\nProcessing coffee shops for spatial matching...\n")
print("="*80)

# Create point geometries from lat/lon
coffee_shops['geometry'] = coffee_shops.apply(
    lambda row: Point(row['location/lng'], row['location/lat'])
    if pd.notna(row['location/lat']) and pd.notna(row['location/lng'])
    else None,
    axis=1
)

# Create GeoDataFrame with WGS84 (lat/lon coordinate system)
gdf_coffee = gpd.GeoDataFrame(
    coffee_shops,
    geometry='geometry',
    crs='EPSG:4326'  # lat/lon coordinate system
)

# Remove records without coordinates
gdf_coffee = gdf_coffee[gdf_coffee['geometry'].notna()]

print(f"✓ Created {len(gdf_coffee)} point geometries")

# Convert to the same CRS as statistical areas (EPSG:2039)
print(f"\nConverting coordinate systems...")
print(f"  Coffee shops CRS: {gdf_coffee.crs} (WGS84 lat/lon)")
print(f"  Statistical areas CRS: {gdf_areas.crs} (Israeli TM Grid)")

gdf_coffee_transformed = gdf_coffee.to_crs(gdf_areas.crs)
print(f"✓ Coffee shops converted to {gdf_coffee_transformed.crs}")

print("\n" + "="*80)
print("Performing spatial join (matching coffee shops to statistical areas)...")

# Spatial join
coffee_result = gpd.sjoin(
    gdf_coffee_transformed,
    gdf_areas[['STAT_2022', 'geometry']],
    how='left',
    predicate='within'
)

# Count results
matched = coffee_result['STAT_2022'].notna().sum()
unmatched = coffee_result['STAT_2022'].isna().sum()

print(f"\n✓✓✓ Spatial join complete!")
print(f"\n  ✓ Matched: {matched} coffee shops")
print(f"  ✗ Unmatched: {unmatched} coffee shops")

if unmatched > 0:
    print(f"\n⚠ Warning: {unmatched} coffee shops could not be matched")
    print("   (These might be outside Eilat's statistical area boundaries)")

print("\n" + "="*80)
print("Distribution of coffee shops by statistical area:")
distribution = coffee_result['STAT_2022'].value_counts().sort_index()
print(distribution)

print("\n" + "="*80)
print("Summary statistics:")
if matched > 0:
    print(f"  Total coffee shops: {len(coffee_result)}")
    print(f"  Statistical areas with coffee shops: {coffee_result['STAT_2022'].nunique()}")
    print(f"  Average coffee shops per area: {matched / coffee_result['STAT_2022'].nunique():.1f}")
    print(f"  Max coffee shops in one area: {distribution.max()}")
    print(f"  Min coffee shops in one area: {distribution.min()}")

# Check which statistical areas have NO coffee shops
all_stat_areas = set(gdf_areas['STAT_2022'].values)
areas_with_coffee = set(coffee_result['STAT_2022'].dropna().values)
empty_areas = sorted(all_stat_areas - areas_with_coffee)

if empty_areas:
    print(f"\n⚠ Statistical areas with NO coffee shops: {empty_areas}")
else:
    print(f"\n✓ All statistical areas have at least one coffee shop")


Processing coffee shops for spatial matching...

✓ Created 339 point geometries

Converting coordinate systems...
  Coffee shops CRS: EPSG:4326 (WGS84 lat/lon)
  Statistical areas CRS: EPSG:2039 (Israeli TM Grid)
✓ Coffee shops converted to EPSG:2039

Performing spatial join (matching coffee shops to statistical areas)...

✓✓✓ Spatial join complete!

  ✓ Matched: 337 coffee shops
  ✗ Unmatched: 2 coffee shops

⚠ Warning: 2 coffee shops could not be matched
   (These might be outside Eilat's statistical area boundaries)

Distribution of coffee shops by statistical area:
STAT_2022
11.0    131
12.0     30
13.0     51
14.0     10
15.0      8
16.0     52
17.0      2
18.0      5
19.0      2
21.0      2
24.0      2
25.0      1
31.0     13
33.0     16
42.0      2
43.0      1
44.0      1
45.0      2
46.0      5
47.0      1
Name: count, dtype: int64

Summary statistics:
  Total coffee shops: 339
  Statistical areas with coffee shops: 20
  Average coffee shops per area: 16.9
  Max coffee shops i

In [8]:
print("Creating map with ALL coffee shops (matched + unmatched)...\n")

# Convert back to WGS84 for Folium
gdf_areas_wgs84 = gdf_areas.to_crs('EPSG:4326')

# Calculate center from all coffee shops
center_lat = coffee_result['location/lat'].mean()
center_lon = coffee_result['location/lng'].mean()

print(f"Map center: {center_lat:.6f}, {center_lon:.6f}")

# Create base map
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=13,
    tiles='OpenStreetMap'
)

# Color scheme for areas
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred',
          'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white',
          'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray',
          'yellow', 'lightpink', 'olive', 'teal', 'navy', 'brown']

print("Adding statistical areas...")

# Add statistical areas
for idx, row in gdf_areas_wgs84.iterrows():
    stat_code = int(row['STAT_2022'])
    color = colors[idx % len(colors)]

    coffee_count = len(coffee_result[coffee_result['STAT_2022'] == stat_code])

    popup_text = f"""
    <b>Statistical Area: {stat_code}</b><br>
    Coffee Shops: {coffee_count}<br>
    Area: {row['Shape__Are']:.0f} m²
    """

    folium.GeoJson(
        row.geometry,
        style_function=lambda x, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 2,
            'fillOpacity': 0.3
        },
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(m)

    # Add label
    centroid = row.geometry.centroid
    folium.Marker(
        location=[centroid.y, centroid.x],
        icon=folium.DivIcon(html=f"""
            <div style="
                font-size: 12px;
                font-weight: bold;
                color: black;
                background-color: white;
                border: 2px solid black;
                border-radius: 5px;
                padding: 2px 5px;
            ">{stat_code}</div>
        """)
    ).add_to(m)

# Separate matched and unmatched
matched_coffee = coffee_result[coffee_result['STAT_2022'].notna()]
unmatched_coffee = coffee_result[coffee_result['STAT_2022'].isna()]

print(f"Adding {len(matched_coffee)} MATCHED coffee shops (BLUE)...")

# Add MATCHED coffee shops as BLUE markers
for idx, row in matched_coffee.iterrows():
    popup_text = f"""
    <b>{row['title']}</b><br>
    Category: {row['categoryName']}<br>
    Score: {row['totalScore']}<br>
    Address: {row['street']}<br>
    <b>✓ Statistical Area: {int(row['STAT_2022'])}</b>
    """

    folium.CircleMarker(
        location=[row['location/lat'], row['location/lng']],
        radius=6,
        popup=folium.Popup(popup_text, max_width=300),
        color='blue',
        fill=True,
        fillColor='blue',
        fillOpacity=0.7,
        weight=2
    ).add_to(m)

print(f"Adding {len(unmatched_coffee)} UNMATCHED coffee shops (RED)...")

# Add UNMATCHED coffee shops as RED markers
for idx, row in unmatched_coffee.iterrows():
    popup_text = f"""
    <b>{row['title']}</b><br>
    Category: {row['categoryName']}<br>
    Score: {row['totalScore']}<br>
    Address: {row['street']}<br>
    <b>✗ Outside statistical areas</b>
    """

    folium.CircleMarker(
        location=[row['location/lat'], row['location/lng']],
        radius=6,
        popup=folium.Popup(popup_text, max_width=300),
        color='red',
        fill=True,
        fillColor='red',
        fillOpacity=0.9,
        weight=2
    ).add_to(m)

# Add legend
legend_html = '''
<div style="position: fixed;
            bottom: 50px; right: 50px; width: 200px; height: 120px;
            background-color: white; border:2px solid grey; z-index:9999;
            font-size:14px; padding: 10px">
<p><strong>Coffee Shops Legend</strong></p>
<p><span style="color:blue;">⬤</span> Matched (in statistical area)</p>
<p><span style="color:red;">⬤</span> Unmatched (outside boundaries)</p>
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

# Save map
print("\nSaving map...")
m.save('eilat_coffee_shops_all_map.html')
print("✓ Map saved as: eilat_coffee_shops_all_map.html")

files.download('eilat_coffee_shops_all_map.html')

print("\n✓✓✓ Map created!")
print(f"\nMap shows:")
print(f"  - BLUE dots ({len(matched_coffee)}): Coffee shops matched to statistical areas")
print(f"  - RED dots ({len(unmatched_coffee)}): Coffee shops outside boundaries")
print(f"  - Total: {len(coffee_result)} coffee shops")

# Display in notebook
m

Creating map with ALL coffee shops (matched + unmatched)...

Map center: 29.555194, 34.954150
Adding statistical areas...
Adding 337 MATCHED coffee shops (BLUE)...
Adding 2 UNMATCHED coffee shops (RED)...

Saving map...
✓ Map saved as: eilat_coffee_shops_all_map.html


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✓✓✓ Map created!

Map shows:
  - BLUE dots (337): Coffee shops matched to statistical areas
  - RED dots (2): Coffee shops outside boundaries
  - Total: 339 coffee shops


In [9]:
print("Manually assigning unmatched coffee shops to statistical area 11...\n")
print("="*80)

# Show the unmatched coffee shops first
unmatched = coffee_result[coffee_result['STAT_2022'].isna()]
print("Unmatched coffee shops:")
print(unmatched[['cid', 'title', 'street', 'location/lat', 'location/lng']])

# Assign stat_2022 = 11 to unmatched coffee shops
coffee_result.loc[coffee_result['STAT_2022'].isna(), 'STAT_2022'] = 11

# Verify the assignment
print("\n" + "="*80)
print("After manual assignment:")
matched = coffee_result['STAT_2022'].notna().sum()
unmatched_count = coffee_result['STAT_2022'].isna().sum()

print(f"  ✓ Matched: {matched} coffee shops")
print(f"  ✗ Unmatched: {unmatched_count} coffee shops")

print("\n" + "="*80)
print("Updated distribution:")
distribution = coffee_result['STAT_2022'].value_counts().sort_index()
print(distribution)

print("\n✓✓✓ All coffee shops now have a statistical area!")
print(f"Statistical area 11 now has {distribution[11.0]} coffee shops (including 2 manually assigned)")

Manually assigning unmatched coffee shops to statistical area 11...

Unmatched coffee shops:
                      cid                                title  \
61   16610670765719936151                       קיסוס - Kissos   
189  15022544564624736051  ספינת אוריוניה הפלגות ארועים מסיבות   

                        street  location/lat  location/lng  
61   Derech Pa'amei HaShalom 1     29.547652     34.953467  
189               HaMayim St 1     29.548390     34.959984  

After manual assignment:
  ✓ Matched: 339 coffee shops
  ✗ Unmatched: 0 coffee shops

Updated distribution:
STAT_2022
11.0    133
12.0     30
13.0     51
14.0     10
15.0      8
16.0     52
17.0      2
18.0      5
19.0      2
21.0      2
24.0      2
25.0      1
31.0     13
33.0     16
42.0      2
43.0      1
44.0      1
45.0      2
46.0      5
47.0      1
Name: count, dtype: int64

✓✓✓ All coffee shops now have a statistical area!
Statistical area 11 now has 133 coffee shops (including 2 manually assigned)


In [10]:
print("Creating map with ALL coffee shops matched (all BLUE)...\n")

# Convert back to WGS84 for Folium
gdf_areas_wgs84 = gdf_areas.to_crs('EPSG:4326')

# Calculate center
center_lat = coffee_result['location/lat'].mean()
center_lon = coffee_result['location/lng'].mean()

print(f"Map center: {center_lat:.6f}, {center_lon:.6f}")

# Create base map
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=13,
    tiles='OpenStreetMap'
)

# Color scheme for areas
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred',
          'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white',
          'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray',
          'yellow', 'lightpink', 'olive', 'teal', 'navy', 'brown']

print("Adding statistical areas...")

# Add statistical areas
for idx, row in gdf_areas_wgs84.iterrows():
    stat_code = int(row['STAT_2022'])
    color = colors[idx % len(colors)]

    coffee_count = len(coffee_result[coffee_result['STAT_2022'] == stat_code])

    popup_text = f"""
    <b>Statistical Area: {stat_code}</b><br>
    Coffee Shops: {coffee_count}<br>
    Area: {row['Shape__Are']:.0f} m²
    """

    folium.GeoJson(
        row.geometry,
        style_function=lambda x, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 2,
            'fillOpacity': 0.4
        },
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(m)

    # Add label
    centroid = row.geometry.centroid
    folium.Marker(
        location=[centroid.y, centroid.x],
        icon=folium.DivIcon(html=f"""
            <div style="
                font-size: 12px;
                font-weight: bold;
                color: black;
                background-color: white;
                border: 2px solid black;
                border-radius: 5px;
                padding: 2px 5px;
            ">{stat_code}</div>
        """)
    ).add_to(m)

print(f"Adding {len(coffee_result)} coffee shops (all matched - BLUE)...")

# Add ALL coffee shops as BLUE markers (all are now matched)
for idx, row in coffee_result.iterrows():
    popup_text = f"""
    <b>{row['title']}</b><br>
    Category: {row['categoryName']}<br>
    Score: {row['totalScore']}<br>
    Address: {row['street']}<br>
    <b>Statistical Area: {int(row['STAT_2022'])}</b>
    """

    if pd.notna(row['url']):
        popup_text += f"<br><a href='{row['url']}' target='_blank'>View Details</a>"

    folium.CircleMarker(
        location=[row['location/lat'], row['location/lng']],
        radius=6,
        popup=folium.Popup(popup_text, max_width=300),
        color='blue',
        fill=True,
        fillColor='blue',
        fillOpacity=0.7,
        weight=2
    ).add_to(m)

# Save map
print("\nSaving map...")
m.save('eilat_coffee_shops_final_map.html')
print("✓ Map saved as: eilat_coffee_shops_final_map.html")

files.download('eilat_coffee_shops_final_map.html')

print("\n✓✓✓ Final map created!")
print(f"\nMap shows:")
print(f"  - All {len(coffee_result)} coffee shops are now matched (BLUE)")
print(f"  - 2 coffee shops manually assigned to area 11")
print(f"  - Click on dots for coffee shop details")
print(f"  - Click on areas for statistics")

# Display in notebook
m

Creating map with ALL coffee shops matched (all BLUE)...

Map center: 29.555194, 34.954150
Adding statistical areas...
Adding 339 coffee shops (all matched - BLUE)...

Saving map...
✓ Map saved as: eilat_coffee_shops_final_map.html


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✓✓✓ Final map created!

Map shows:
  - All 339 coffee shops are now matched (BLUE)
  - 2 coffee shops manually assigned to area 11
  - Click on dots for coffee shop details
  - Click on areas for statistics


In [11]:
print("Preparing final coffee shops dataset for export...\n")
print("="*80)

# Create final dataframe with stat_2022 column
final_coffee = coffee_shops.copy()

# Add stat_2022 column from the spatial join result
# Match by cid to ensure correct alignment
final_coffee['stat_2022'] = None

for idx, row in coffee_result.iterrows():
    final_coffee.loc[final_coffee['cid'] == row['cid'], 'stat_2022'] = row['STAT_2022']

# Convert stat_2022 to integer
final_coffee['stat_2022'] = final_coffee['stat_2022'].astype(int)

# Reorder columns - put stat_2022 after coordinates
columns_order = [
    'cid',
    'title',
    'description',
    'categoryName',
    'totalScore',
    'temporarilyClosed',
    'permanentlyClosed',
    'url',
    'website',
    'street',
    'location/lat',
    'location/lng',
    'stat_2022',  # New foreign key column
    'Activity times'
]

final_coffee = final_coffee[columns_order]

print("Final dataset preview:")
print(final_coffee.head(10))

print("\n" + "="*80)
print("Summary:")
print(f"✓ Total coffee shops: {len(final_coffee)}")
print(f"✓ Columns: {len(final_coffee.columns)}")
print(f"✓ New 'stat_2022' column added (foreign key to statistical areas)")
print(f"✓ All coffee shops have statistical areas (2 manually assigned to area 11)")

# Show distribution
print("\n" + "="*80)
print("Distribution of coffee shops by statistical area:")
print(final_coffee['stat_2022'].value_counts().sort_index())

# Export to Excel
print("\n" + "="*80)
print("Exporting files...")

excel_filename = 'Eilat_coffeeShops_with_stat_2022.xlsx'
final_coffee.to_excel(excel_filename, index=False, engine='openpyxl')
print(f"✓ Excel file created: {excel_filename}")

# Export to CSV (with UTF-8 encoding)
csv_filename = 'Eilat_coffeeShops_with_stat_2022.csv'
final_coffee.to_csv(csv_filename, index=False, encoding='utf-8-sig')
print(f"✓ CSV file created: {csv_filename}")

print("\n" + "="*80)
print("Downloading files...")

# Download both files
files.download(excel_filename)
files.download(csv_filename)

print("\n✓✓✓ SUCCESS! Files downloaded!")
print("\nYou now have:")
print(f"  1. Excel file: {excel_filename} ({len(final_coffee)} coffee shops)")
print(f"  2. CSV file: {csv_filename} ({len(final_coffee)} coffee shops)")
print(f"  3. Interactive map: eilat_coffee_shops_final_map.html")
print("\nAll files include the 'stat_2022' foreign key column!")
print("Note: 2 coffee shops were manually assigned to statistical area 11")

Preparing final coffee shops dataset for export...

Final dataset preview:
                    cid                                  title description  \
0  13304296502578382367            Ilan's The Alternative Cafe         NaN   
1  12043325052248165199              Cafe Orca - Marina Divers         NaN   
2  10214898044927479307                           DESERT & SEA         NaN   
3   2125374989964811871         Haoman 9- Coffee Confectionery         NaN   
4   6409751934090735763               Eilat wines | יינות אילת         NaN   
5  10391316705344374484                 Uga Chaga Bakery Eilat         NaN   
6    273177842584156848                    COFFEE BAR SCORPION         NaN   
7   8637314661383373859  Café LOBBY at Shoshana קפה לובי שושנה         NaN   
8  16582584323208051397                       בייקרי קופי מרקט         NaN   
9  15185075342429951591                          Crema Express         NaN   

  categoryName  totalScore  temporarilyClosed  permanentlyClosed  

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✓✓✓ SUCCESS! Files downloaded!

You now have:
  1. Excel file: Eilat_coffeeShops_with_stat_2022.xlsx (339 coffee shops)
  2. CSV file: Eilat_coffeeShops_with_stat_2022.csv (339 coffee shops)
  3. Interactive map: eilat_coffee_shops_final_map.html

All files include the 'stat_2022' foreign key column!
Note: 2 coffee shops were manually assigned to statistical area 11
